In [ ]:
%matplotlib inline
import scipy.special, cmath, scipy.stats
import random, math, pylab, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pylab
import mpl_toolkits.mplot3d
from random import gauss
from math import exp, log, sqrt

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Option Parameters
# Initial price of the underlying
S0 = 100.0
# Constant annual risk-free rate
r = 0.05
# Constant annual volatility
sigma = 0.2
# Time to maturity
T = 1.0
# Strike price of the option
K = 105.0

# Black-Scholes formula

In [ ]:
def d_j(j, S, K, r, v, T):
    return (log(S/K) + (r + ((-1)**(j-1))*0.5*v*v)*T)/(v*(T**0.5))

def vanilla_call_price(S, K, r, v, T):
    return S*scipy.stats.norm.cdf(d_j(1, S, K, r, v, T))-K*exp(-r*T) * scipy.stats.norm.cdf(d_j(2, S, K, r, v, T))

def vanilla_put_price(S, K, r, v, T):
    return -S*scipy.stats.norm.cdf(-d_j(1, S, K, r, v, T))+K*exp(-r*T) * scipy.stats.norm.cdf(-d_j(2, S, K, r, v, T))

In [ ]:
print vanilla_call_price(S0, K, r, sigma, T)

# Binomial tree approach

In [ ]:
# Time Parameters
# Number of time steps
M_binomial = 1000
# Discrete time interval
dt_binomial = T / M_binomial
# discount factor per time interval
df = math.exp(-r * dt_binomial)

In [ ]:
# Binomial Parameters
u = math.exp(sigma * math.sqrt(dt_binomial))  # up-movement
d = 1 / u  # down-movement
q = (math.exp(r * dt_binomial) - d) / (u - d)  # risk-adjusted synthetic up probability

## Efficient implementation: numpy

In [ ]:
def binom_np(initial_value, nb_steps, strike):
    # Index Levels with NumPy
    mu = np.arange(nb_steps + 1)
    mu = np.resize(mu, (nb_steps + 1, nb_steps + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = initial_value * mu * md
    # Valuation Loop
    V = np.maximum(S - strike, 0)
    Qu = np.zeros((nb_steps + 1, nb_steps + 1), dtype=np.float64)
    Qu[:, :] = q  
    Qd = 1 - Qu 
    z = 0
    for t in range(nb_steps - 1, -1, -1):  # backwards iteration
        V[0:nb_steps - z, t] = (Qu[0:nb_steps - z, t] * V[0:nb_steps - z, t + 1]
                                + Qd[0:nb_steps - z, t] * V[1:nb_steps - z + 1, t + 1]) * df
        z += 1
    return V[0, 0]

In [ ]:
%time C = binom_np(S0, M_binomial, K)
round(C, 3)

## More elementary approach: EXERCISE!

In [ ]:
def binom_py(initial_value, nb_steps, strike):
    # LOOP 1 - Index Levels
    S = np.zeros((nb_steps + 1, nb_steps + 1), dtype=np.float64)  # index level array
    S[0, 0] = initial_value
    z1 = 0
    for j in xrange(1, nb_steps + 1, 1):
        z1 += 1
        for i in xrange(z1 + 1):
            S[i, j] = S[0, 0] * (u ** j) * (d ** (i * 2)) 
            
    # LOOP 2 - Inner Values
    iv = np.zeros((nb_steps + 1, nb_steps + 1), dtype=np.float64)  # inner value array
    # TODO Fill in the iv array
    
    # LOOP 3 - Valuation
    pv = np.zeros((nb_steps + 1, nb_steps + 1), dtype=np.float64)  # present value array
    # TODO Fill in the pv array
    
    return pv[0, 0]

## Check your result

In [ ]:
%time C = binom_py(S0, M_binomial, K)
round(C, 3)

# Monte Carlo approach

In [ ]:
# Use smaller number of steps in Monte Carlo....otherwise booom
M_MC = 100
dt_MC = T / M_MC

## Elementary implementation

In [ ]:
# Elementary simulation of I paths with M time steps
def genS_py(I, S0, nb_steps):
    ''' I : number of world lines '''
    S = []
    # S is to be an array of arrays
    # each array in S is a path for the security in a given world line
    # we need to append I such paths into S and return it
    # use z = gauss(0.0, 1.0) to generate the Wiener process
    
    for i in range(I):
        # GENERATE A PATH path
        S.append(path)
    return S

In [ ]:
I = 100000
%time S = genS_py(I, S0, M_MC)

In [ ]:
for i in range(20):
    plt.plot(S[i])
plt.xlabel('Time Steps')
plt.ylabel('Index Level')
plt.grid(True)

## Note: the paths can be used for **any** payoff!

In [ ]:
C0 = math.exp(-r * T) * sum([max(path[-1] - K, 0) for path in S]) / I
round(C0, 3)

## Vectorized implementation with numpy

In [ ]:
# Simulation of I paths with M time steps using advanced numpy functionality
def genS_np(I, S0, nb_steps):
    print S0
    ''' I : number of index level paths '''
    S = S0 * np.exp(np.cumsum((r - 0.5 * sigma ** 2) * dt_MC
                + sigma * np.sqrt(dt_MC) * np.random.standard_normal((I, nb_steps + 1)), axis=1))
    return S

In [ ]:
%time S2 = genS_np(I, S0, M_MC)

In [ ]:
for i in range(20):
    plt.plot(S2[i])
plt.xlabel('Time Steps')
plt.ylabel('Index Level')
plt.grid(True)

In [ ]:
C0 = math.exp(-r * T) * sum([max(path[-1] - K, 0) for path in S2]) / I
round(C0, 3)